In [1]:
from aam.data_handlers.generator_dataset import GeneratorDataset

2025-01-29 16:50:39.269675: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-29 16:50:39.269724: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-29 16:50:39.269754: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 16:50:39.279993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf

In [3]:
# gpus = tf.config.list_physical_devices("GPU")
# if len(gpus) > 0:
#     tf.config.experimental.set_memory_growth(gpus[0], True)
# gpus

In [4]:
import pandas as pd
fecal = pd.read_csv('fecal.tsv', sep='\t')

fecal_filtered = fecal[['sample_name', 'covid_positive']]

fecal_filtered

def check_covid_positive(row):
    if row =='yes':
        return 1
    else:
        return 0

fecal_filtered['has_covid'] = fecal_filtered['covid_positive'].apply(check_covid_positive)
fecal_filtered.set_index('sample_name', inplace=True)
fecal_filtered

/tmp/ipykernel_3061/4195698998.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fecal_filtered['has_covid'] = fecal_filtered['covid_positive'].apply(check_covid_positive)


,covid_positive,has_covid
sample_name,,
14812.N.066,no,0
14812.N.051,no,0
14812.N.068,no,0
14812.N.037,no,0
14812.CV10.R025,recovered,0
14812.CV.19.R069,recovered,0
14812.CV.19.R071,recovered,0
14812.CV19.R.022,recovered,0
14812.CV.19.R078,recovered,0


In [5]:
import tensorflow as tf
class ASVWrapper(tf.keras.layers.Layer):
    def __init__(self, asv_encoder, **kwargs):
        super(ASVWrapper, self).__init__(**kwargs)
        self.asv_encoder = asv_encoder
        self.asv_encoder.trainable = False
        self.embedding_dim = self.asv_encoder.embedding_dim
    def call(self, inputs, training=False):
        embeddings, counts = self.asv_encoder(inputs, return_asv_embeddings=True, training=False)
        return embeddings, counts
    def get_config(self):
        config = super().get_config()
        config.update({"asv_encoder": tf.keras.saving.serialize_keras_object(self.asv_encoder)})
    @classmethod
    def from_config(cls, config):
        config["asv_encoder"] = tf.keras.saving.deserialize_keras_object(config["asv_encoder"])
        model = cls(**config)
        return model

In [6]:
def get_dataset(gen):
    enqueuer = tf.keras.utils.OrderedEnqueuer(gen, use_multiprocessing=True)
    enqueuer.start(workers=2, max_queue_size=gen.steps_per_epoch)
    gen.stop = lambda: enqueuer.stop(0.1)
    y_type = tf.TensorSpec(shape=(gen.samples_per_minibatch, 1), dtype=tf.string)
    dataset = tf.data.Dataset.from_generator(
        enqueuer.get,
        output_signature=(
            (
                tf.TensorSpec(shape=[None, 150], dtype=tf.int32),
                tf.TensorSpec(shape=[None, 2], dtype=tf.int32),
                tf.TensorSpec(shape=[None], dtype=tf.int32),
                tf.TensorSpec(shape=[None, 1], dtype=tf.int32),
            ),
            tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
        ),
    )
    return dataset

In [7]:
gd = GeneratorDataset(
    table='/home/swchan/DSC170/fecal.biom',
    metadata=fecal_filtered,
    metadata_column='has_covid',
    shuffle=False,
    is_categorical=False,
    shift=0,
    scale=1
)

aligning table with metadata
aligned table shape: (7969, 60)
aligned metadata shape: (60,)
done preprocessing metadata
rarefy table...
removing empty sample/obs from table
creating encoder target...
encoder target created


In [8]:
x, y = gd[0]
print(y.shape)
a, b, c, d = x
a.shape, b.shape, c.shape, d.shape

(8, 1)


((1205, 150), (1845, 2), (1845,), (1845, 1))

In [9]:
dataset = get_dataset(gd)

2025-01-29 16:50:43.981272: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
# import pandas as pd
# from biom import load_table
# df = ""

# df["covid_positive"]

# table = load_table("fecal.biom")
# df = df.sort_index(table.ids())

In [11]:
# import tensorflow as tf
# model = tf.keras.models.load_model('/home/swchan/DSC170/model.keras', compile=False)
# model = ASVWrapper(model)

# @tf.function(reduce_retracing=True)
# def call_step(inputs):
#     return model(inputs)
                 
# for x, y in dataset.take(gd.steps_per_epoch):
#     embeddings, counts = call_step(x)
#     attention_mask = tf.cast(counts  >  0, dtype=tf.float32)
#     attention_mask = tf.matmul(attention_mask, attention_mask, transpose_b=True)
#     print("k")
# gd.stop()

#Output B x A x E Tensor (Sample Number x ASVs x Embedding Dimension)
# B x A x E Tensor (Sample Number x ASVs x 1)

In [12]:
import tensorflow as tf
base_model = tf.keras.models.load_model('/home/swchan/DSC170/model.keras', compile=False)
base_model = ASVWrapper(base_model)

/home/swchan/.conda/envs/aam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reconstructing ASVEncoder...
create asv layer with 8 heads
Using linear bias
Encoder residual connection...
Encoder exit... True
ASVEncoder exit... True
Using linear bias
Pooler residual connection...
Pooler exit... True
BaseSequenceEncoder exit... True
Constructing UnifracDenoser from config
Encoder residual connection...
Encoder exit... False
ASVEncoder exit... False
Pooler residual connection...
Pooler exit... False
BaseSequenceEncoder exit... False
Input Shape in build: (None, None, 128)
Using linear bias
Encoder residual connection...
Encoder exit... True
Using linear bias
Pooler residual connection...
Pooler exit... True
UnifracEncoder exit... True
Input Shape in build: (None, None, 128)
Using linear bias
Encoder residual connection...
Encoder exit... True
Using linear bias
Pooler residual connection...
Pooler exit... True
UnifracEncoder exit... True
UniFracDenoiser exit... True
UnifracDenoiser is already built


In [13]:
# x, y = gd[0]
# embeddings, count = base_model(x)
#Output B x A x E Tensor (Sample Number x ASVs x Embedding Dimension)
# B x A x E Tensor (Sample Number x ASVs x 1)

In [14]:
K = tf.keras

In [15]:
# class TransformerEncodingBlock(K.layers.Layer):
#     def __init__(self, num_attention_heads=4, dropout_rate=0.1, **kwargs):
#         super().__init__(**kwargs)
#         self.num_attention_heads = num_attention_heads
#         self.dropout_rate = dropout_rate

#     def build(self, input_shape):
#         emb_dim = input_shape[-1]
#         key_dim = emb_  adim // self.num_attention_heads
#         self.attention_layer = K.layers.MultiHeadAttention(self.num_attention_heads, key_dim=key_dim, dropout=self.dropout_rate)
#         self.attention_layer._build_from_signature(input_shape, input_shape)
#         linear_bias = LinearBiasSoftmax()
#         setattr(self.attention_layer, 'softmax', linear_bias)
        
#         self.ff_inner = K.layers.Dense(512, activation="gelu")
#         self.ff = K.layers.Dense(emb_dim, activation="gelu")
#         self.rezero_alpha = self.add_weight("rezero_alpha", 1, dtype="float32", initializer="zero", trainable=True)

#     def call(self, input, attention_mask=None, training=False):
#         """
#         input: [B, A, N] float
#         attention_mask: [B, A, 1] bool
#         """
#         # attention
#         attention_mask = tf.matmul(attention_mask, attention_mask, transpose_b=True)  # [B, A, A]
#         attention_output = self.attention_layer(input, input, attention_mask=attention_mask, training=training)

#         # rezero skip connection
#         attention_output = input + self.rezero_alpha * attention_output

#         ff_output = self.ff_inner(attention_output)
#         ff_output = self.ff(ff_output)

#         # rezero skip connection
#         ff_output = attention_output + self.rezero_alpha * ff_output

#         return ff_output

In [16]:
# class TransformerEncoder(K.layers.Layer):
#     def __init__(self, num_attention_layers=4, num_attention_heads=4, dropout_rate=0.1, **kwargs):
#         super().__init__(**kwargs)
#         self.num_attention_layers = num_attention_layers
#         self.num_attention_heads = num_attention_heads
#         self.dropout_rate = dropout_rate

#         self.encoding_layers = [
#             TransformerEncodingBlock(self.num_attention_heads, self.dropout_rate) for _ in range(self.num_attention_layers)
#         ]
#         self.rezero_alpha = self.add_weight("rezero_alpha", 1, dtype="float32", initializer="zero", trainable=True)

#     def call(self, inputs, attention_mask=None, training=False):
#         output = inputs
#         for i in range(self.num_attention_layers):
#             output = self.encoding_layers[i](output, attention_mask=attention_mask, training=training)
#         return inputs * self.rezero_alpha * output

In [17]:
K=tf.keras
from aam.models.transformers import TransformerEncoder
from aam.models.multihead_attention_pooling import MultiHeadAttentionPooling
class Classifier(K.Model):
    def __init__(self, feature_extractor, **kwargs):
        super().__init__(**kwargs)

        self.feature_extractor = feature_extractor
        self.feature_extractor.trainable = False

        self.encoder = TransformerEncoder(
            num_layers=4, 
            num_attention_heads=4, 
            intermediate_size=512, 
            dropout_rate=0.1, 
            normalize_outputs=False,
            use_residual_connections=True,
            use_linear_bias=False
        )  # transformer (or anything else)
        self.pooling = MultiHeadAttentionPooling(normalize_output=False,  use_linear_bias=False)  # extracts sample embeddings from our ASV embeddings
        self.dense_ff = K.layers.Dense(1)  # sample_embedings => classifys covid +/-

        self.loss_fn = K.losses.BinaryCrossentropy(from_logits=True)

        self.auc_tracker = K.metrics.Mean()
        self.loss_tracker = K.metrics.Mean()

    def call(self, inputs, mask=None, training=False):
        """
        inputs: [B, A, N], B: batch_dim, A: # ASV in sample, N: nuctides,
        string tensor
        """


        # aam case
        features, counts = self.feature_extractor(inputs)
        mask =  tf.cast(counts >  0, dtype="float32")

        encodig_output = self.encoder(features, mask=mask, training=training)  # [B, A, N]
        pooling_output = self.pooling(encodig_output, mask=mask, training=training)  # [B, N]
        return self.dense_ff(pooling_output)

    def train_step(self, data):
        x, y = data

        # create attention mask
        # example [["ACTG"], [""]]
        with tf.GradientTape() as tape:
            output = self(x, training=True)
            loss = self.loss_fn(y, output)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(loss)
        self.auc_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result(), "auc": self.auc_tracker.result()}

    def test_step(self, data):
        x, y = data
        output = self(x, training=False)
        loss = self.loss_fn(y, output)

        self.loss_tracker.update_state(loss)
        self.auc_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result(), "auc": self.auc_tracker.result()}

    def predict_step(self, data):
        x, y = data
        return self(x)

In [18]:
model = Classifier(base_model)

optimizer = K.optimizers.Adam(
    learning_rate=1e-4
)

In [19]:
x, y = gd[0]
model(x)

Encoder residual connection...
Encoder exit... False
ASVEncoder exit... False
Pooler residual connection...
Pooler exit... False
BaseSequenceEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
UniFracDenoiser exit... False
Using linear bias
Encoder residual connection...
Encoder exit... True
Pooler residual connection...
Pooler exit... True


<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[-0.10795963],
       [ 0.06111354],
       [-0.10168678],
       [-0.18776545],
       [-0.03257203],
       [-0.24220324],
       [ 0.06716362],
       [-0.1405243 ]], dtype=float32)>

In [20]:
print(y)

[[0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]]


In [21]:
# model.build([None, None, 1])
model.compile(optimizer=optimizer, run_eagerly=False)

In [22]:
# core_callbacks = [
#     tf.keras.callbacks.EarlyStopping(
#         "val_loss",
#         patience=25,
#         start_from_epoch=25
#     )
]
model.fit(dataset, epochs=1, steps_per_epoch=gd.steps_per_epoch) 

Encoder residual connection...
Encoder exit... False
ASVEncoder exit... False
Pooler residual connection...
Pooler exit... False
BaseSequenceEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
UniFracDenoiser exit... False
Encoder residual connection...
Encoder exit... True
Pooler residual connection...
Pooler exit... True
Encoder residual connection...
Encoder exit... False
ASVEncoder exit... False
Pooler residual connection...
Pooler exit... False
BaseSequenceEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
Encoder residual connection...
Encoder exit... False
Pooler residual connection...
Pooler exit... False
UnifracEncoder exit... False
UniFracDen

In [23]:
"ASVWrapper" in str(type(base_model))

True

In [ ]:
#Test, Train, Valid Sets